In [1]:
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, LayerNormalization, BatchNormalization
import tensorflow as tf

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA

In [2]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive//My Drive/Andmeteadus/Masinõpe/4N/")

Mounted at /content/drive


In [3]:
data = pd.read_csv("cleaned_with_title_am_nam.csv", sep="\t")
print(data.shape)
data.dropna()
train_, test_ = train_test_split(data, train_size=0.8, test_size=0.2, shuffle=True)
def preprocess(data):
  new_articles = []
  for i in data.iterrows():
    if type(i[1]["title"]) == float or type(i[1]["article"]) == float:
      new_articles.append("")
      continue

    new_articles.append(i[1]["title"]+" "+i[1]["article"])
  data["data"] = np.array(new_articles)
  return data
#data.head()

(1218, 4)


In [4]:
le = LabelEncoder()
train = preprocess(train_)
train_X = train["data"]
train_y_ = train["label"]
train_y = to_categorical(le.fit_transform(train_y_))

test = preprocess(test_)
test_X = test["data"]
test_y_ = test["label"]
test_y = to_categorical(le.transform(test_y_))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [5]:
test_y.shape, test_X.shape, train_X.shape, train_y.shape

((244, 2), (244,), (974,), (974, 2))

In [16]:
tfidf_vect = TfidfVectorizer(stop_words="english")
X_train_tfidf = tfidf_vect.fit_transform(train_X)

X_test_tfidf = tfidf_vect.transform(test_X)


vect_size = X_train_tfidf.shape[1]

model = Sequential()
#model.add(Dense(1024,input_shape=(vect_size,), activation="tanh"))
#model.add(BatchNormalization())
#model.add(Dropout(0.5))
#model.add(Dense(512,input_shape=(vect_size,), activation="relu"))
#model.add(LayerNormalization())
#model.add(Dropout(0.5))
model.add(Dense(128,input_shape=(vect_size,), activation="tanh"))
model.add(LayerNormalization())
model.add(Dropout(0.5))
model.add(Dense(32, activation="relu"))
model.add(LayerNormalization())
model.add(Dropout(0.5))

#0.934
#d128 
#d32
#d0.5, layer

#0.93
#d1024 tanh
#bn
#d0.5
#d256 tanh
#bn
#d0.5
#d128 tanh
#bn
#d0.5
#d32 relu
#bn
#d0.5
model.add(Dense(2, activation='softmax'))

model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 128)               4235136   
_________________________________________________________________
layer_normalization_5 (Layer (None, 128)               256       
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 32)                4128      
_________________________________________________________________
layer_normalization_6 (Layer (None, 32)                64        
_________________________________________________________________
dropout_6 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 2)                

In [17]:
model.fit(X_train_tfidf.toarray(), train_y,
          batch_size=64,
          epochs=20,
          validation_data=(X_test_tfidf.toarray(), test_y))

Epoch 1/20
16/16 [==============================] - 1s 50ms/step - loss: 0.7025 - accuracy: 0.7341 - val_loss: 0.3119 - val_accuracy: 0.8607
Epoch 2/20
16/16 [==============================] - 1s 38ms/step - loss: 0.2549 - accuracy: 0.9035 - val_loss: 0.2964 - val_accuracy: 0.8852
Epoch 3/20
16/16 [==============================] - 1s 37ms/step - loss: 0.1042 - accuracy: 0.9630 - val_loss: 0.3341 - val_accuracy: 0.8934
Epoch 4/20
16/16 [==============================] - 1s 38ms/step - loss: 0.0645 - accuracy: 0.9815 - val_loss: 0.3658 - val_accuracy: 0.8852
Epoch 5/20
16/16 [==============================] - 1s 39ms/step - loss: 0.0526 - accuracy: 0.9815 - val_loss: 0.3919 - val_accuracy: 0.8811
Epoch 6/20
16/16 [==============================] - 1s 38ms/step - loss: 0.0380 - accuracy: 0.9897 - val_loss: 0.4585 - val_accuracy: 0.8811
Epoch 7/20
16/16 [==============================] - 1s 38ms/step - loss: 0.0236 - accuracy: 0.9938 - val_loss: 0.5079 - val_accuracy: 0.8852
Epoch 8/20
16

In [18]:
y_predict = model.predict(X_test_tfidf.toarray())
y_pred = np.argmax(y_predict, axis=1)
y_true = np.argmax(test_y, axis=1)

metrics.f1_score(y_true, y_pred)

0.894736842105263

In [19]:
pca = PCA(n_components=0.95)
X_train = pca.fit_transform(X_train_tfidf.toarray())
X_test = pca.transform(X_test_tfidf.toarray())
vect_size = X_train.shape[1]
X_test.shape, X_train.shape

((244, 821), (974, 821))

In [27]:
model = Sequential()
#model.add(Dense(1024,input_shape=(vect_size,), activation="tanh"))
#model.add(LayerNormalization())
#model.add(Dropout(0.5))
model.add(Dense(256, input_shape=(vect_size,), activation="tanh"))
model.add(LayerNormalization())
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(LayerNormalization())
model.add(Dropout(0.25))
model.add(Dense(16, activation="relu"))
model.add(LayerNormalization())
model.add(Dropout(0.25))

model.add(Dense(2, activation='softmax'))

model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 256)               210432    
_________________________________________________________________
layer_normalization_11 (Laye (None, 256)               512       
_________________________________________________________________
dropout_11 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 64)                16448     
_________________________________________________________________
layer_normalization_12 (Laye (None, 64)                128       
_________________________________________________________________
dropout_12 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 16)               

In [28]:
model.fit(X_train, train_y,
          batch_size=64,
          epochs=40,
          validation_data=(X_test, test_y))

Epoch 1/40
16/16 [==============================] - 0s 20ms/step - loss: 1.0846 - accuracy: 0.5339 - val_loss: 0.5850 - val_accuracy: 0.7049
Epoch 2/40
16/16 [==============================] - 0s 7ms/step - loss: 0.7025 - accuracy: 0.6314 - val_loss: 0.4420 - val_accuracy: 0.8115
Epoch 3/40
16/16 [==============================] - 0s 7ms/step - loss: 0.6495 - accuracy: 0.6817 - val_loss: 0.3878 - val_accuracy: 0.8279
Epoch 4/40
16/16 [==============================] - 0s 7ms/step - loss: 0.5164 - accuracy: 0.7515 - val_loss: 0.3432 - val_accuracy: 0.8648
Epoch 5/40
16/16 [==============================] - 0s 7ms/step - loss: 0.4300 - accuracy: 0.7916 - val_loss: 0.3074 - val_accuracy: 0.9016
Epoch 6/40
16/16 [==============================] - 0s 8ms/step - loss: 0.3771 - accuracy: 0.8439 - val_loss: 0.2952 - val_accuracy: 0.8852
Epoch 7/40
16/16 [==============================] - 0s 7ms/step - loss: 0.2717 - accuracy: 0.8932 - val_loss: 0.2853 - val_accuracy: 0.8934
Epoch 8/40
16/16 [=

In [29]:
y_predict = model.predict(X_test)
y_pred = np.argmax(y_predict, axis=1)
y_true = np.argmax(test_y, axis=1)

metrics.f1_score(y_true, y_pred)

0.8859649122807017